In [1]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm 
import pprint as pp

In [2]:
TRAIN_DIR = 'train'#'X:/Kaggle_Data/dogs_vs_cats/train/train'
TEST_DIR = 'test'#'X:/Kaggle_Data/dogs_vs_cats/test/test'
IMG_SIZE = 64
LR = 1e-3
train_path = "/Users/julesgarrett/Downloads/tiny-imagenet-200/train"
label_path = "/Users/julesgarrett/Downloads/tiny-imagenet-200"

In [3]:
train_data =[]
label = []
counter = 0
for file in tqdm(os.listdir(train_path)):
    if counter == 20:
        break
    for pic in os.listdir(train_path+"/"+file+"/images"):
        img = cv2.imread(train_path+"/"+file+"/images/"+pic)
        train_data.append(np.array(img))
        label.append(file)
    counter += 1
train_data = np.array(train_data)
# np.save('tiny-imagenet-train.npy', train_data)

 10%|█         | 20/200 [00:06<01:02,  2.89it/s]


In [4]:
label = np.array(label)

label = np.reshape(label, (len(label), 1))


In [5]:
lab_id = {}
lab = []
seen = []
labels = []
counter = 0
for l in tqdm(range(label.shape[0])):
    if counter == 20:
        break
    if label[l] in seen:
        continue
    else:
        seen.append(label[l])
        l_id = str(label[l])
        new_dict_entry = {l_id : counter}
        counter += 1 
        lab_id.update(new_dict_entry)
# print(lab_id) 
num_classes = counter
print(num_classes)

  0%|          | 0/10000 [00:00<?, ?it/s]

20


In [6]:
for l in tqdm(range(label.shape[0])):
    temp = np.zeros(num_classes)
    clss = label[l]
    index = lab_id.get(str(clss))
    temp[index] = 1
    labels.append(temp)

100%|██████████| 10000/10000 [00:00<00:00, 45755.73it/s]


In [7]:
labels = np.array(labels)
# print(labels.shape)

In [8]:
training_data = []
for l in tqdm(range(label.shape[0])):
    training_data.append([train_data[l], labels[l]])
    
shuffle(training_data)

np.save('training_data.npy', training_data)


100%|██████████| 10000/10000 [00:00<00:00, 455190.14it/s]


In [9]:

trn_data = np.load('training_data.npy')
print(trn_data.shape)

(10000, 2)


In [10]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')
#       net = layers.conv2d(inputs, 64, [11, 11], 4, padding='VALID', scope='conv1')
#       net = layers_lib.max_pool2d(net, [3, 3], 2, scope='pool1')

#       net = layers.conv2d(net, 192, [5, 5], scope='conv2')
#       net = layers_lib.max_pool2d(net, [3, 3], 2, scope='pool2')

#       net = layers.conv2d(net, 384, [3, 3], scope='conv3')
#       net = layers.conv2d(net, 384, [3, 3], scope='conv4')
#       net = layers.conv2d(net, 256, [3, 3], scope='conv5')
#       net = layers_lib.max_pool2d(net, [3, 3], 2, scope='pool5')
convnet = conv_2d(convnet, 64, [11,11],  4, activation='relu')
convnet = max_pool_2d(convnet,[3,3], 2)

convnet = conv_2d(convnet, 192, [5,5], activation='relu')
convnet = max_pool_2d(convnet,[3,3], 2)

convnet = conv_2d(convnet, 384, [3,3], activation='relu')

convnet = conv_2d(convnet, 384, [3,3], activation='relu')

convnet = conv_2d(convnet, 256, [3,3], activation='relu')
convnet = max_pool_2d(convnet,[3,3], 2)

convnet = fully_connected(convnet, 1024, activation='relu')

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, num_classes, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [11]:
# print(trn_data)
train = trn_data[:-500]
test = trn_data[-500:]

In [12]:
x = np.array([i[0] for i in train])
# print(x.shape)
y = [i[1] for i in train]

test_x = np.array([i[0] for i in test])
test_y = [i[1] for i in test]

In [13]:
MODEL_NAME = 'test_cnn-{}-{}.model'.format(LR, '2conv-basic')
model.fit({'input': x}, {'targets': y}, n_epoch=10, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 1489  | total loss: 1.49465 | time: 74.309s
| Adam | epoch: 010 | loss: 1.49465 - acc: 0.5411 -- iter: 9472/9500
Training Step: 1490  | total loss: 1.48808 | time: 75.841s
| Adam | epoch: 010 | loss: 1.48808 - acc: 0.5386 | val_loss: 2.23884 - val_acc: 0.4080 -- iter: 9500/9500
--
